In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow_hub as hub
import shutil

df = pd.read_csv("/content/drive/MyDrive/Capstone/dataset.csv")

In [ ]:
df.isna().sum()

In [ ]:
embedded_tittle_column = hub.text_embedding_column(key='tittle',
                                                   module_spec='',
                                                   trainable=False)

content_id_column = tf.feature_column.categorical_column_with_hash_bucket(key='content_id',
                                                                          hash_bucket_size = len(content_ids_list)+1)
embedded_content_column = tf.feature_column.embedding_column(categorical_column = content_id_column,
                                                             dimension = 10)

author_column = tf.feature_column.categorical_column_with_hash_bucket(key='author',
                                                                          hash_bucket_size = len(authors_list)+1)
embedded_author_column = tf.feature_column.embedding_column(categorical_column = content_id_column,
                                                             dimension = 3)

category_column_categorical = tf.feature_column.categorical_column_with_vocabulary_list(key='category',
                                                                                        vocabulary_list=categories_list,
                                                                                        num_oov_bucket = 1)

category_column = tf.feature_column.indicator_column(category_column_categorical)

months_since_epoch_boundaries = list(range(400,700,20))
months_since_epoch_column = tf.feature_column.numeric.column(key = 'months_since_epoch')
months_since_epoch_bucketized = tf.feature_column.bucketized_column(source_column = months_since_epoch_column,
                                                                    boundaries = months_since_epoch_boudaries)

crossed_month_since_category_column = tf.feature_column.indicator_column(tf.feature_column.crossed_column(keys = [categorical_column_categorical, months_since_epoch_bucketized],
                                                                       hash_bucket_sized = len(months_since_epoch_boundaries)*(len(categories_list+1))))

feature_columns = [embedded_content_column,
                   embedded_author_column,
                   category_column,
                   embedded_title_column,
                   crossed_month_since_category_colum]

In [ ]:
record_defaults = [['Unknowwn'], ['Unknown'], ['Unknown'], ['Unknown'], [mean_months_since_epoch], ['Unknown']]
column_keys = ['visitor_id', 'content_id', 'category', 'title', 'author', 'months_since_epoch', 'next_content_id']
label_key = 'next_content_id'

def read_dataset(filename, node, batch_size = 512):
  def _input_fn():
    def decode_csv(value_column):
      columns = tf.decode_csv(value_column.record_defaults = record_defaults)
      features = dict(zip(column_keys, columns))
      label = features.pop(label_key)
      return features, label

    file_list = tf.gfile.Glob(filename)

    dataset = tf.data.TextLineDataset(filelist).map(decode_csv)

    if node == tf.estimator.Nodekeys.TRAIN:
      num_epoch = None
      dataset = dataset.shuffle(buffer_size = 10 = batch_size)
    else:
      num_epoch = 1
    
    dataset = dataset.repeat(num_epoch).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

  return _input_fn


In [ ]:
def model_fn(features, labels, node, params):
  net = tf.feature_column.input_layer(features, params['feature_columns'])
  for units in params['hidden_units']:
    net = tf.layers.dense(net, units=units, activation = 'relu')

  logits = tf.layer.dense(net, param['n_classes'], activation = None)

  prediction_classes = tf.argmax(logits, 1)
  from tensorflow.python.lib.io import file_io

  with file_io.FileIO('content_ids.txt', node='r') as ifp:
    content = tf.constant([x.rstrip() for x in ifp])
  predicted_class_names = tf.gather(content, predicted_classes)
  if node == tf.estimator.NodeKeys.PEEDICT:
    predictions = (
        'class_ids': prediction_classes[:, tf.newaxis],
        'class_names': prediction_class_names[:, tf.newaxis],
        'probabilities': tf.nn.softmax(logits),
        'logits': logits,
    )
    return tf.EstimatorSpec(node, predictions=predictions)
  table = tf.contrib.lookup.index_table_from_file(vocabulary_file='content_ids.txt')
  labels = table.lookup(labels)

  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  accuracy = tf.metrics.accuracy(labels=labels,
                                 predictions=predicted_classes,
                                 name='acc_op')
  top_10_accuracy = tf.metrics.mean(tf.nn.in_top_k(predictions=logits, targets=labels, k=10))

  metrics = (
      'accuracy': accuracy,
      'top_10_accuracy': top_10_accuracy
  )

  tf.summary.scalar('accuracy',accuracy[1])
  tf.summary.scalar('top_10_accuracy', top_10_accuracy[1])

  if node == tf.estimator.NodeKeys.EVAL:
    return tf.estimator.EstimatorSpec(
        node, loss=loss, eval_metrics_ops=metrics)
  
  assert node == tf.estimator.NodeKeys.TRAIN

  optimizer = tf.trainAdagradOptimizer(learning_rate=0.1)
  trains_op = optimizer.minimize(loss, global_step= tf.train.get_global_step())
  return tf.estimator.EstimatorSpec(node, loss=loss, train_op=train_op)

In [ ]:
outdir = 'content_based_model_trained'
shutil.retree(outdir, ignore_errors = True)
estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    model_dir=outdir,
    params=(
        'feature_colums':feature_columns,
        'hidden_units': [200,100,50],
        'n_classes': len(content_ids_list)
    ))

train_spec = tf.estimator.TrainSpec(
    input_fn = read.dataset('training_set.csv', tf.estimator.NodeKeys.TRAIN),
    max_steps = 2000)

eval_spec = tf.estimator.EvalSpec(
    input_fn = read_dataset('test_set.csv', tf.estimator.NodeKeys.EVAL),
    steps = None,
    start_delay_secs = 30,
    throttle_secs = 60)

tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

Adjusment Code

In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow_hub as hub
import shutil

In [ ]:
# Membaca dataset
df = pd.read_csv("/content/drive/MyDrive/Capstone/dataset.csv")

In [ ]:
# Menginisialisasi kolom fitur
embedded_title_column = hub.text_embedding_column(key='Name',
                                                  module_spec='')

calories_column = tf.feature_column.numeric_column(key='Calories')

feature_columns = [embedded_title_column, calories_column]

In [ ]:
# Menentukan label key (rekomendasi makanan)
label_key = 'food_recommendation'

In [ ]:
# Mengatur default values dan keys untuk kolom-kolom dataset
record_defaults = [['RecipeId'], [0]]
column_keys = ['Name', 'Calories']

In [ ]:
# Fungsi untuk membaca dataset
def read_dataset(filename, mode, batch_size=512):
    def _input_fn():
        def decode_csv(value_column):
            columns = tf.decode_csv(value_column, record_defaults=record_defaults)
            features = dict(zip(column_keys, columns))
            label = features.pop(label_key)
            return features, label

        dataset = tf.data.TextLineDataset(filename).map(decode_csv)

        if mode == tf.estimator.ModeKeys.TRAIN:
            dataset = dataset.shuffle(buffer_size=10 * batch_size)

        dataset = dataset.repeat().batch(batch_size)
        return dataset

    return _input_fn

In [ ]:
outdir = 'content_based_model_trained'
shutil.rmtree(outdir, ignore_errors=True)

In [ ]:
# Model Function
def model_fn(features, labels, mode, params):
    # Mendefinisikan layer input menggunakan feature columns
    input_layer = tf.feature_column.input_layer(features, params['feature_columns'])

    # Mendefinisikan hidden layers
    hidden_layer1 = tf.layers.dense(inputs=input_layer, units=200, activation=tf.nn.relu)
    hidden_layer2 = tf.layers.dense(inputs=hidden_layer1, units=100, activation=tf.nn.relu)
    hidden_layer3 = tf.layers.dense(inputs=hidden_layer2, units=50, activation=tf.nn.relu)

    # Mendefinisikan output layer (rekomendasi makanan)
    logits = tf.layers.dense(inputs=hidden_layer3, units=params['n_classes'])

    # Menghasilkan prediksi jika mode adalah PREDICT
    predictions = {
        'food_recommendation': tf.argmax(logits, axis=1),
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Menghitung loss
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Menghitung accuracy
    accuracy = tf.metrics.accuracy(labels=labels, predictions=predictions['food_recommendation'])

    # Konfigurasi metrik evaluasi
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    # Konfigurasi metode evaluasi dan training
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)

    optimizer = tf.train.AdamOptimizer()
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

    # Konfigurasi metode training
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [ ]:
# Membuat Estimator
estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    model_dir=outdir,
    params={
        'feature_columns': feature_columns,
        'hidden_units': [200, 100, 50],
        'n_classes': len(df['food_recommendation'].unique())
    })

In [ ]:
# Spesifikasi pelatihan
train_spec = tf.estimator.TrainSpec(
    input_fn=read_dataset('dataset.csv', tf.estimator.ModeKeys.TRAIN),
    max_steps=2000)

In [ ]:
# Spesifikasi evaluasi
eval_spec = tf.estimator.EvalSpec(
    input_fn=read_dataset('dataset.csv', tf.estimator.ModeKeys.EVAL),
    steps=None,
    start_delay_secs=30,
    throttle_secs=60)

In [ ]:
# Melatih dan mengevaluasi model
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
